<h1>Amazon apparel Recommendations</h1>

### SOUHARDYA ADHIKARY
You can see my profile at [GITHUB](https://github.com/SOUHARDYAADHIKARY1999)

### Connect me at my linkedIn profile
https://www.linkedin.com/in/souhardya-adhikary-a10a10156/


### [1.0] Import the packages

In [6]:
import pandas as pd

In [7]:
# we have give a json file which consists of all information about
# the products
# loading the data using pandas' read_json file.
data=pd.read_json(open('/Users\DELL\PythonProjects\RECOMENDATION_ENGINE/tops_fashion.json',"r",encoding="utf-8"))

In [8]:
print(data)

         sku        asin product_type_name formatted_price author  \
0       None  B016I2TS4W             SHIRT            None   None   
1       None  B01N49AI08             SHIRT            None   None   
2       None  B01JDPCOHO             SHIRT            None   None   
3       None  B01N19U5H5             SHIRT            None   None   
4       None  B004GSI2OS             SHIRT          $26.26   None   
...      ...         ...               ...             ...    ...   
183133  None  B01MSALTSO         OUTERWEAR          $14.58   None   
183134  None  B015W98YQK             SHIRT            None   None   
183135  None  B075756PGC             SHIRT            None   None   
183136  None  B074L8FVTT             SHIRT          $44.99   None   
183137  None  B074FTYJQC             SHIRT            None   None   

                    color          brand      publisher  \
0                    None          FNC7C           None   
1                    None   FIG Clothing           No

In [9]:
print ('Number of data points : ', data.shape[0])
print ('Number of features/variables:', data.shape[1])

Number of data points :  183138
Number of features/variables: 19


### Terminology
Let's see the attributes we hav

In [10]:
# each product/item has 19 features in the raw dataset.
print(data.columns)# prints column-names or feature-names.

Index(['sku', 'asin', 'product_type_name', 'formatted_price', 'author',
       'color', 'brand', 'publisher', 'availability', 'reviews',
       'large_image_url', 'availability_type', 'small_image_url',
       'editorial_review', 'title', 'model', 'medium_image_url',
       'manufacturer', 'editorial_reivew'],
      dtype='object')
